In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
import re

def setup_driver():
    """Configurar el driver de Selenium"""
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # Comenta para ver el navegador
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
    chrome_options.add_argument("--window-size=1920,1080")
    
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def hacer_scroll_completo(driver):
    """Hacer scroll completo para cargar todas las propiedades"""
    print("🔄 Haciendo scroll completo...")
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    intentos = 0
    max_intentos = 5
    
    while intentos < max_intentos:
        # Scroll al final
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Scroll adicional suave
        for i in range(2):
            driver.execute_script(f"window.scrollBy(0, 500);")
            time.sleep(0.5)
        
        # Calcular nueva altura
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            intentos += 1
        else:
            intentos = 0
            
        last_height = new_height
        
        if intentos >= 2:  # Si no hay cambio después de 2 intentos, salir
            break

def encontrar_y_hacer_click_pagina(driver, numero_pagina):
    """Encontrar y hacer click en un número de página específico"""
    print(f"🔍 Buscando página {numero_pagina}...")
    
    # Lista de selectores para probar
    selectors = [
        f"//a[text()='{numero_pagina}']",
        f"//button[text()='{numero_pagina}']",
        f"//span[text()='{numero_pagina}']",
        f"//*[contains(@class, 'page') and text()='{numero_pagina}']",
        f"//*[contains(@class, 'pagination')]//a[text()='{numero_pagina}']",
        f"//*[contains(@class, 'pagination')]//button[text()='{numero_pagina}']",
        f"//*[contains(@class, 'pagination')]//span[text()='{numero_pagina}']",
    ]
    
    for selector in selectors:
        try:
            elemento = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, selector))
            )
            print(f"✅ Encontrada página {numero_pagina} con selector: {selector}")
            
            # Scroll al elemento
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", elemento)
            time.sleep(1)
            
            # Verificar que no es la página actual
            if elemento.get_attribute("class") and "active" in elemento.get_attribute("class"):
                print(f"ℹ️ La página {numero_pagina} ya está activa")
                return False
            
            # Hacer click
            driver.execute_script("arguments[0].click();", elemento)
            print(f"✅ Click exitoso en página {numero_pagina}")
            return True
        except:
            continue
    
    # Buscar en contenedores de paginación
    contenedores = [
        "//div[contains(@class, 'pagination')]",
        "//nav[contains(@class, 'pagination')]",
        "//ul[contains(@class, 'pagination')]",
        "//div[contains(@class, 'results-pagination')]",
    ]
    
    for contenedor in contenedores:
        try:
            pagination_container = driver.find_element(By.XPATH, contenedor)
            print(f"✅ Encontrado contenedor: {contenedor}")
            
            # Buscar todos los elementos dentro del contenedor
            elementos = pagination_container.find_elements(By.XPATH, ".//*")
            
            for elemento in elementos:
                if elemento.text.strip() == str(numero_pagina):
                    # Verificar que no es la página actual
                    if elemento.get_attribute("class") and "active" in elemento.get_attribute("class"):
                        print(f"ℹ️ La página {numero_pagina} ya está activa")
                        return False
                    
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", elemento)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", elemento)
                    print(f"✅ Click en página {numero_pagina} desde contenedor")
                    return True
        except:
            continue
    
    print(f"❌ No se pudo encontrar la página {numero_pagina}")
    return False

def verificar_pagina_cargada(driver, numero_pagina, timeout=15):
    """Verificar que la página se cargó correctamente"""
    try:
        # Esperar a que carguen las propiedades
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CLASS_NAME, "property-card__container"))
        )
        
        # Verificar que hay propiedades
        propiedades = driver.find_elements(By.CLASS_NAME, "property-card__container")
        if propiedades:
            print(f"✅ Página {numero_pagina} cargada correctamente ({len(propiedades)} propiedades)")
            return True
        
        print(f"❌ Página {numero_pagina} cargada pero sin propiedades")
        return False
        
    except TimeoutException:
        print(f"❌ Timeout esperando página {numero_pagina}")
        return False

def extraer_datos_propiedad(card):
    """Extraer datos detallados de una propiedad"""
    try:
        data = {}
        
        # Titulo
        try:
            titulo_elem = card.find_element(By.CLASS_NAME, "property-card__detail-title")
            data['titulo'] = titulo_elem.text.strip()
        except:
            data['titulo'] = ""
        
        # Ubicación
        try:
            ubicacion_elem = card.find_element(By.CLASS_NAME, "property-card__detail-top__left")
            data['ubicacion'] = ubicacion_elem.text.strip()
        except:
            data['ubicacion'] = ""
        
        # Precio
        try:
            precio_elem = card.find_element(By.CLASS_NAME, "property-card__detail-price")
            data['precio'] = precio_elem.text.strip()
        except:
            data['precio'] = ""
        
        # Características
        try:
            specs_elem = card.find_element(By.CLASS_NAME, "property-card__detail-specs")
            specs_text = specs_elem.text.strip()
            data['caracteristicas'] = specs_text
            
            # Extraer valores específicos
            area_match = re.search(r'(\d+)\s*m²', specs_text)
            data['area_m2'] = area_match.group(1) if area_match else ""
            
            hab_match = re.search(r'(\d+)\s*hab', specs_text)
            data['habitaciones'] = hab_match.group(1) if hab_match else ""
            
            bano_match = re.search(r'(\d+)\s*bañ', specs_text)
            data['banos'] = bano_match.group(1) if bano_match else ""
            
            parq_match = re.search(r'(\d+)\s*par', specs_text)
            data['parqueaderos'] = parq_match.group(1) if parq_match else ""
            
        except:
            data['caracteristicas'] = ""
            data['area_m2'] = ""
            data['habitaciones'] = ""
            data['banos'] = ""
            data['parqueaderos'] = ""
        
        # URL
        try:
            link_elem = card.find_element(By.TAG_NAME, "a")
            data['url'] = link_elem.get_attribute("href")
            if data['url']:
                codigo_match = re.search(r'/([A-Z0-9-]+)$', data['url'])
                data['codigo'] = codigo_match.group(1) if codigo_match else ""
            else:
                data['codigo'] = ""
        except:
            data['url'] = ""
            data['codigo'] = ""
        
        return data if data['titulo'] else None
        
    except Exception as e:
        print(f"Error extrayendo propiedad: {e}")
        return None

def scrape_pagina(driver, numero_pagina):
    """Extraer todas las propiedades de una página específica"""
    print(f"\n📄 PROCESANDO PÁGINA {numero_pagina}")
    print("-" * 50)
    
    start_time = time.time()
    
    # Hacer scroll completo
    hacer_scroll_completo(driver)
    time.sleep(2)
    
    # Encontrar todas las propiedades
    try:
        property_cards = driver.find_elements(By.CLASS_NAME, "property-card__container")
        print(f"📊 Encontradas {len(property_cards)} propiedades")
    except:
        print("❌ No se pudieron encontrar propiedades")
        return []
    
    propiedades_pagina = []
    propiedades_extraidas = 0
    
    for i, card in enumerate(property_cards, 1):
        try:
            propiedad = extraer_datos_propiedad(card)
            if propiedad:
                propiedad['pagina'] = numero_pagina
                propiedad['numero_en_pagina'] = i
                propiedades_pagina.append(propiedad)
                propiedades_extraidas += 1
                
                # Mostrar progreso cada 10 propiedades
                if i % 10 == 0:
                    print(f"   📦 Extraídas {i}/{len(property_cards)} propiedades...")
                    
        except Exception as e:
            continue
    
    end_time = time.time()
    tiempo_pagina = end_time - start_time
    
    print(f"✅ Página {numero_pagina} completada: {propiedades_extraidas} propiedades ({tiempo_pagina:.1f}s)")
    
    return propiedades_pagina

def scrape_100_paginas(max_paginas=100):
    """Scraping automático de hasta 100 páginas"""
    driver = setup_driver()
    todas_propiedades = []
    
    try:
        # Página 1 - Navegación inicial
        url = "https://www.metrocuadrado.com/inmuebles/arriendo/medellin/"
        print(f"🌐 Accediendo a página 1: {url}")
        driver.get(url)
        
        # Esperar carga inicial
        if not verificar_pagina_cargada(driver, 1):
            print("❌ Error cargando página 1")
            return []
        
        pagina_actual = 1
        paginas_fallidas = 0
        max_fallos_consecutivos = 3
        
        while pagina_actual <= max_paginas and paginas_fallidas < max_fallos_consecutivos:
            print(f"\n{'='*60}")
            print(f"🎯 PÁGINA ACTUAL: {pagina_actual}")
            print(f"📈 PROGRESO: {pagina_actual}/{max_paginas}")
            print(f"📊 TOTAL EXTRAÍDO: {len(todas_propiedades)} propiedades")
            print(f"{'='*60}")
            
            # Extraer página actual
            propiedades_pagina = scrape_pagina(driver, pagina_actual)
            
            if propiedades_pagina:
                todas_propiedades.extend(propiedades_pagina)
                paginas_fallidas = 0  # Resetear contador de fallos
                
                # Guardar progreso cada 5 páginas
                if pagina_actual % 5 == 0:
                    guardar_progreso(todas_propiedades, pagina_actual)
            else:
                paginas_fallidas += 1
                print(f"❌ Fallo en página {pagina_actual} ({paginas_fallidas}/{max_fallos_consecutivos})")
            
            # Intentar navegar a la siguiente página (excepto en la última iteración)
            if pagina_actual < max_paginas:
                siguiente_pagina = pagina_actual + 1
                print(f"\n🔄 Intentando navegar a página {siguiente_pagina}...")
                
                if encontrar_y_hacer_click_pagina(driver, siguiente_pagina):
                    # Esperar carga de nueva página
                    time.sleep(5)
                    
                    if verificar_pagina_cargada(driver, siguiente_pagina):
                        pagina_actual = siguiente_pagina
                    else:
                        paginas_fallidas += 1
                        print(f"❌ Error cargando página {siguiente_pagina}")
                else:
                    paginas_fallidas += 1
                    print(f"❌ No se pudo encontrar página {siguiente_pagina}")
            else:
                print("✅ Límite de páginas alcanzado")
                break
            
            # Pequeña pausa entre páginas para no sobrecargar el servidor
            time.sleep(2)
        
        if paginas_fallidas >= max_fallos_consecutivos:
            print(f"\n⚠️  Detenido por {max_fallos_consecutivos} fallos consecutivos")
                
    except Exception as e:
        print(f"❌ Error fatal: {e}")
    finally:
        driver.quit()
    
    return todas_propiedades

def guardar_progreso(propiedades, pagina_actual):
    """Guardar progreso temporal"""
    if propiedades:
        df = pd.DataFrame(propiedades)
        archivo_temp = f"progreso_pagina_{pagina_actual}.xlsx"
        df.to_excel(archivo_temp, index=False)
        print(f"💾 Progreso guardado: {archivo_temp}")

def guardar_resultados_finales(propiedades):
    """Guardar resultados finales"""
    if not propiedades:
        print("❌ No hay datos para guardar")
        return
    
    df = pd.DataFrame(propiedades)
    
    # Estadísticas
    print(f"\n{'='*50}")
    print("📊 ESTADÍSTICAS FINALES")
    print(f"{'='*50}")
    print(f"Total propiedades: {len(propiedades):,}")
    
    if 'pagina' in df.columns:
        paginas = df['pagina'].unique()
        print(f"Páginas procesadas: {len(paginas)}")
        print(f"Rango de páginas: {min(paginas)} - {max(paginas)}")
        
        # Propiedades por página
        for pagina in sorted(paginas)[:10]:  # Mostrar primeras 10 páginas
            count = len(df[df['pagina'] == pagina])
            print(f"  Página {pagina}: {count} propiedades")
        
        if len(paginas) > 10:
            print(f"  ... y {len(paginas) - 10} páginas más")
    
    # Guardar en múltiples formatos
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    archivo_base = f"propiedades_metrocuadrado_{timestamp}"
    
    formats = {
        'excel': f'{archivo_base}.xlsx',
        'csv': f'{archivo_base}.csv',
        'json': f'{archivo_base}.json'
    }
    
    for formato, archivo in formats.items():
        try:
            if formato == 'excel':
                df.to_excel(archivo, index=False)
            elif formato == 'csv':
                df.to_csv(archivo, index=False, encoding='utf-8-sig')
            elif formato == 'json':
                df.to_json(archivo, orient='records', indent=2, force_ascii=False)
            print(f"✅ Guardado: {archivo}")
        except Exception as e:
            print(f"❌ Error guardando {archivo}: {e}")

def main():
    print("🚀 INICIANDO SCRAPING DE 100 PÁGINAS")
    print("⏰ Este proceso puede tomar varias horas...")
    print("💡 Recomendado: Ejecutar en headless mode para mejor rendimiento")
    print("🛑 Puedes detener en cualquier momento con Ctrl+C\n")
    
    start_time = time.time()
    
    try:
        # Scraping de hasta 100 páginas
        propiedades = scrape_100_paginas(max_paginas=100)
        
        end_time = time.time()
        tiempo_total = end_time - start_time
        horas = int(tiempo_total // 3600)
        minutos = int((tiempo_total % 3600) // 60)
        segundos = int(tiempo_total % 60)
        
        print(f"\n🎉 SCRAPING COMPLETADO!")
        print(f"⏱️  Tiempo total: {horas:02d}:{minutos:02d}:{segundos:02d}")
        
        if propiedades:
            guardar_resultados_finales(propiedades)
        else:
            print("❌ No se extrajeron propiedades")
            
    except KeyboardInterrupt:
        print(f"\n🛑 Scraping interrumpido por el usuario")
        print(f"📊 Propiedades extraídas hasta el momento: {len(propiedades) if 'propiedades' in locals() else 0}")
        if 'propiedades' in locals() and propiedades:
            guardar_resultados_finales(propiedades)

if __name__ == "__main__":
    main()

🚀 INICIANDO SCRAPING DE 100 PÁGINAS
⏰ Este proceso puede tomar varias horas...
💡 Recomendado: Ejecutar en headless mode para mejor rendimiento
🛑 Puedes detener en cualquier momento con Ctrl+C

🌐 Accediendo a página 1: https://www.metrocuadrado.com/inmuebles/arriendo/medellin/
✅ Página 1 cargada correctamente (9 propiedades)

🎯 PÁGINA ACTUAL: 1
📈 PROGRESO: 1/100
📊 TOTAL EXTRAÍDO: 0 propiedades

📄 PROCESANDO PÁGINA 1
--------------------------------------------------
🔄 Haciendo scroll completo...
📊 Encontradas 68 propiedades
   📦 Extraídas 10/68 propiedades...
   📦 Extraídas 20/68 propiedades...
   📦 Extraídas 30/68 propiedades...
   📦 Extraídas 40/68 propiedades...
   📦 Extraídas 50/68 propiedades...
   📦 Extraídas 60/68 propiedades...
✅ Página 1 completada: 68 propiedades (18.3s)

🔄 Intentando navegar a página 2...
🔍 Buscando página 2...
✅ Encontrada página 2 con selector: //a[text()='2']
✅ Click exitoso en página 2
✅ Página 2 cargada correctamente (9 propiedades)

🎯 PÁGINA ACTUAL: 2
📈

In [27]:
df_a = pd.read_csv('propiedades_metrocuadrado_20250925_170602.csv', encoding='utf-8-sig')
df_a.duplicated().sum()


np.int64(0)

In [17]:
df_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5325 entries, 0 to 5324
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   titulo            5325 non-null   object 
 1   ubicacion         5325 non-null   object 
 2   precio            5325 non-null   object 
 3   caracteristicas   5324 non-null   object 
 4   area_m2           5324 non-null   float64
 5   habitaciones      4424 non-null   float64
 6   banos             5059 non-null   float64
 7   parqueaderos      4379 non-null   float64
 8   url               5325 non-null   object 
 9   codigo            5325 non-null   object 
 10  pagina            5325 non-null   int64  
 11  numero_en_pagina  5325 non-null   int64  
dtypes: float64(4), int64(2), object(6)
memory usage: 499.3+ KB


In [20]:
df_a[df_a['url'].duplicated()]

,titulo,ubicacion,precio,caracteristicas,area_m2,habitaciones,banos,parqueaderos,url,codigo,pagina,numero_en_pagina
68,"Apartamento en Arriendo, El Poblado, Medellín",El Poblado | Medellín,$15.000.000,270 m²\n4 hab.\n5 bañ.\n2 par.,270.0,4.0,5.0,2.0,https://www.metrocuadrado.com/inmueble/arriend...,19797-M6051464,2,1
136,"Bodega en Arriendo, Cristo Rey, Medellín",Cristo Rey | Suroccidente | Medellín,$3.980.000,165 m²\n2 bañ.,165.0,NaN,2.0,NaN,https://www.metrocuadrado.com/inmueble/arriend...,737-M5859490,3,2
294,"Apartamento en Arriendo, El Poblado, Medellín",El Poblado | Nororiente | Medellín,$4.000.000,90 m²\n3 hab.\n2 bañ.\n2 par.,90.0,3.0,2.0,2.0,https://www.metrocuadrado.com/inmueble/arriend...,16477-M5979653,6,1
296,"Apartamento en Arriendo, Santa Monica, Medellín",Santa Monica | Medellín,$2.300.000,100 m²\n3 hab.\n2 bañ.\n1 par.,100.0,3.0,2.0,1.0,https://www.metrocuadrado.com/inmueble/arriend...,17143-M5627340,6,3
401,"Apartamento en Arriendo, Belen, Medellín",Belen | Suroccidente | Medellín,$2.400.000,85 m²\n3 hab.\n2 bañ.,85.0,3.0,2.0,NaN,https://www.metrocuadrado.com/inmueble/arriend...,2511-M5053633,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...
5220,"Apartamento en Arriendo, Medellin, Medellín",Medellin | Medellín,$3.400.000,85 m²\n3 hab.\n2 bañ.,85.0,3.0,2.0,NaN,https://www.metrocuadrado.com/inmueble/arriend...,11813-M5965769,99,2
5221,"Apartamento en Arriendo, El Poblado, Medellín",El Poblado | Nororiente | Medellín,$4.000.000,90 m²\n3 hab.\n2 bañ.\n2 par.,90.0,3.0,2.0,2.0,https://www.metrocuadrado.com/inmueble/arriend...,16477-M5979653,99,3
5272,"Apartamento en Arriendo, Poblado, Medellín",Poblado | Medellín,$25.000.000,306 m²\n3 hab.\n4 bañ.\n2 par.,306.0,3.0,4.0,2.0,https://www.metrocuadrado.com/inmueble/arriend...,17098-M5781507,100,1
5273,"Apartamento en Arriendo, La America, Medellín",La America | Noroccidente | Medellín,$2.350.000,65 m²\n2 hab.\n2 bañ.\n1 par.,65.0,2.0,2.0,1.0,https://www.metrocuadrado.com/inmueble/arriend...,18177-M5963027,100,2


In [ ]:
palma = df_a[df_a['ubicacion'].str.contains('la Palma', case=False, na=False)]
palma

,titulo,ubicacion,precio,caracteristicas,area_m2,habitaciones,banos,parqueaderos,url,codigo,pagina,numero_en_pagina
277,"Apartaestudio en Arriendo, BELEN LA PALMA, Med...",BELEN LA PALMA | Suroccidente | Medellín,$2.800.000,47 m²\n1 hab.\n1 bañ.\n1 par.,47.0,1.0,1.0,1.0,https://www.metrocuadrado.com/inmueble/arriend...,15254-M5661725,5,37
1251,"Casa en Arriendo, Belen La Palma, Medellín",Belen La Palma | Medellín,$6.500.000,250 m²\n5 hab.\n4 bañ.\n4 par.,250.0,5.0,4.0,4.0,https://www.metrocuadrado.com/inmueble/arriend...,17287-M6057940,24,8
1339,"Casa en Arriendo, Belen La Palma, Medellín",Belen La Palma | Medellín,$4.700.000,180 m²\n4 hab.\n4 bañ.\n1 par.,180.0,4.0,4.0,1.0,https://www.metrocuadrado.com/inmueble/arriend...,4660-M5960460,25,43
1659,"Casa en Arriendo, La Palma, Medellín",La Palma | Suroccidente | Medellín,$2.700.000,106 m²\n3 hab.\n2 bañ.,106.0,3.0,2.0,NaN,https://www.metrocuadrado.com/inmueble/arriend...,2214-M6069149,31,45
2590,"Casa en Arriendo, Belen La Palma, Medellín",Belen La Palma | Medellín,$4.700.000,180 m²\n4 hab.\n4 bañ.\n1 par.,180.0,4.0,4.0,1.0,https://www.metrocuadrado.com/inmueble/arriend...,20772-M5993804,49,22
3306,"Apartamento en Arriendo, Belen La Palma, Medellín",Belen La Palma | Medellín,$2.350.000,85 m²\n3 hab.\n2 bañ.\n1 par.,85.0,3.0,2.0,1.0,https://www.metrocuadrado.com/inmueble/arriend...,14059-M5999786,62,49
4300,"Casa en Arriendo, BELEN LA PALMA, Medellín",BELEN LA PALMA | Suroccidente | Medellín,$2.800.000,80 m²\n2 hab.\n2 bañ.,80.0,2.0,2.0,NaN,https://www.metrocuadrado.com/inmueble/arriend...,12116-M5699524,81,36
4788,"Casa en Arriendo, Belen La Palma, Medellín",Belen La Palma | Medellín,$4.500.000,140 m²\n4 hab.\n4 bañ.\n1 par.,140.0,4.0,4.0,1.0,https://www.metrocuadrado.com/inmueble/arriend...,18518-M5959606,90,47
5039,"Apartamento en Arriendo, BELEN LA PALMA, Medellín",BELEN LA PALMA | Suroccidente | Medellín,$2.200.000,48 m²\n2 hab.\n2 bañ.\n1 par.,48.0,2.0,2.0,1.0,https://www.metrocuadrado.com/inmueble/arriend...,12116-M5665258,95,33
